# Practical Lesson 5: Facility Location

### __Problem Description__:
There are $n$ districts in a metropolitan region. The distance between district $i$ and district $j$ is given by $d_{ij}$. The goal is to choose in which districts Emergency Care Units (UPAs) should be installed. Legislation requires that the distance between a district and the nearest UPA must be at most $R$ (ignore units). What is the minimum number of UPAs that must be installed? Model the problem using Integer Linear Programming and solve the shared instance below using python-mip. In your model, consider $D = \{1, ..., n\}$ as the set of districts.

In [61]:
from mip import *

def solve(model):
    model.verbose = 0
    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value}\n")

    print("Solution:")
    for v in model.vars:
      if v.x > 0.00001 and v.name.find("x")!=-1:
        print(v.name, " = ", v.x)

In [62]:
D = [(64, 74), (40, 9), (59, 5), (43, 6), (69, 51),
     (40, 15), (2, 87), (1, 61), (15, 41), (4, 49),
     (25, 82), (10, 49), (93, 95), (98, 3), (88, 65),
     (48, 28), (47, 18), (52, 75), (94, 84), (54, 83),
     (48, 80), (62, 87), (9, 76), (39, 47), (99, 0),
     (83, 21), (35, 13), (32, 6), (31, 59), (48, 48),
     (63, 25), (76, 47), (64, 42), (72, 66), (23, 49),
     (15, 19), (1, 73), (60, 60), (85, 73), (17, 50),
     (62, 47), (92, 68), (75, 62), (58, 9), (32, 42),
     (22, 58), (41, 36), (94, 18), (23, 43), (50, 96),
     (32, 25), (56, 64), (27, 31), (45, 9), (79, 41),
     (55, 69), (7, 57), (88, 31), (7, 40), (42, 2)]

In [63]:
def calc_dists(points):
    n = len(points)
    dists = [[0 for i in range(n)] for j in range(n)]
    for i in range(n):
        for j in range(n):
            x1, y1 = points[i]
            x2, y2 = points[j]
            d = int(np.sqrt((x1 - x2)**2 + (y1 - y2)**2) + 0.5)
            dists[i][j] = d

    return dists

In [64]:
def create_model(n, R, dist): 
    # n is the number of districts
    # R is the maximum distance between a district and a UPA
    # dist is a list of lists, each dist[i] contains the distance from district i+1 to all other districts

    model = Model(sense=MINIMIZE, solver_name=CBC)

    # x[i] := 1 if there is a UPA in district i + 1, 0 otherwise
    x = [model.add_var(var_type=BINARY, name=f"x_{i}") for i in range(n)]

    # y[i] := number of UPAs that cover district i + 1
    y = [model.add_var(var_type=INTEGER, name=f"y_{i}", lb=0.0) for i in range(n)]

    # Objective function
    model.objective = xsum(x[i] for i in range(n))

    # Cover constraints
    for i in range(n):
        model += y[i] >= 1
        model += y[i] == xsum(x[j] for j in range(n) if dist[i][j] <= R)

    return model

In [65]:
dists = calc_dists(D)

n = len(D)
R = 40
# faça para R = 20, 30 e 40

model = cria_modelo(n, R, dists)
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3.0

Solution:
x_2  =  1.0
x_9  =  1.0
x_33  =  1.0
